In [1]:
import hypernetx as hnx
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import sys

## Load tables

In [6]:
dff = pd.read_pickle("biggerTrans.pkl")

In [7]:
dffwos2 = pd.read_pickle("biggerTransNoSars2.pkl")

In [8]:
dffwos = pd.read_pickle("biggerTransNoSars.pkl")

In [9]:
dffwoc = pd.read_pickle("biggerTransNoCov.pkl")

## Build the hypergraph

In [6]:
dff.shape

(9524, 169)

In [34]:
# now we use some (though not all) of the from_dataframe() arguments and
# let the function take care of the dataframe manipulation
Hf = hnx.Hypergraph.from_dataframe(dff, # the whole dataframe, b and p columns
                                                    #columns=human_b_cols, # choose specific columns
                                                    zsc='columns', # other option is 'rows'
                                                    absolute=True, # absolute value after z-score is taken
                                                    lower_thresh=2) # applies the > 2 threshold after zscore and absolute value)

# options that I used the defaults for:
# transpose = False: this will transpose the dataframe after z-score and absolute value, essentially creating the dual hypergraph. Instead we're taking the dual after the fact (below).
# name = None (string): If you want to give the resulting hypergraph a "name" attribute. Not necessary.
# key = None (function which evaluates True or False): This is for more complcated thresholding. If you're just doing z-score > some threshold you don't need to worry about this.
# rows = None (list of row names): If you want to use only a subset of the rows. This is done before taking z-score so your z-score will be relative only to those rows chosen.
# upper_thresh = None (number): You can have a maximum value for the the zscore if you want. You can use both upper_thresh and lower_thresh.

In [35]:
# running example here for large s value because it finishes relatively quickly. 
# Small s values take a long time on these large hypergraphs!
#betcen30 = hnx.s_betweenness_centrality(HfD, s=30)
#clocen30 = hnx.s_harmonic_closeness_centrality(HfD, s=30)

In [36]:
Hf.shape

(8010, 159)

In [37]:
HfD = Hf.dual()
HfD.shape

(159, 8010)

In [10]:
# now we use some (though not all) of the from_dataframe() arguments and
# let the function take care of the dataframe manipulation
Hfwos2 = hnx.Hypergraph.from_dataframe(dffwos2, # the whole dataframe, b and p columns
                                                    #columns=human_b_cols, # choose specific columns
                                                    zsc='columns', # other option is 'rows'
                                                    absolute=True, # absolute value after z-score is taken
                                                    lower_thresh=2) # applies the > 2 threshold after zscore and absolute value)

# options that I used the defaults for:
# transpose = False: this will transpose the dataframe after z-score and absolute value, essentially creating the dual hypergraph. Instead we're taking the dual after the fact (below).
# name = None (string): If you want to give the resulting hypergraph a "name" attribute. Not necessary.
# key = None (function which evaluates True or False): This is for more complcated thresholding. If you're just doing z-score > some threshold you don't need to worry about this.
# rows = None (list of row names): If you want to use only a subset of the rows. This is done before taking z-score so your z-score will be relative only to those rows chosen.
# upper_thresh = None (number): You can have a maximum value for the the zscore if you want. You can use both upper_thresh and lower_thresh.

In [11]:
Hfwos2.shape

(7960, 154)

In [12]:
Hfwos2D = Hfwos2.dual()

In [13]:
Hfwos2D.shape

(154, 7960)

In [14]:
graph = Hfwos2D

In [15]:
import ray

In [16]:
ray.init(num_cpus=5)

2020-05-26 10:56:02,843	INFO resource_spec.py:204 -- Starting Ray with 9.86 GiB memory available for workers and up to 4.93 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-26 10:56:03,259	INFO services.py:1168 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '130.20.113.55',
 'raylet_ip_address': '130.20.113.55',
 'redis_address': '130.20.113.55:23501',
 'object_store_address': '/tmp/ray/session_2020-05-26_10-56-02_822534_34866/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-26_10-56-02_822534_34866/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-05-26_10-56-02_822534_34866'}

In [17]:
@ray.remote
def betweenness(s, graph = graph):
    sbt = hnx.s_betweenness_centrality(graph, s=s)
    sbt = pd.Series(sbt)
    sbt.to_pickle("intermediateNoSars2/" + str(s) + "-betweenness.pkl")
    return(sbt)

@ray.remote
def closeness(s, graph = graph):
    scl = hnx.s_harmonic_closeness_centrality(graph, s=s)
    scl = pd.Series(scl)
    scl.to_pickle("intermediateNoSars2/" + str(s) + "-closeness.pkl")
    return(scl)    

In [18]:
graph = Hfwos2D

In [19]:
Hfwos2Dscl = ray.get([closeness.remote(i) for i in range(6,51)])

(pid=raylet) F0526 11:13:17.742532 136234432 service_based_gcs_client.cc:92]  Check failed: num_attempts < RayConfig::instance().gcs_service_connect_retries() No entry found for GcsServerAddress
(pid=raylet) *** Check failure stack trace: ***
(pid=raylet)     @        0x10626eae2  google::LogMessage::~LogMessage()
(pid=raylet)     @        0x105dcd555  ray::RayLog::~RayLog()
(pid=raylet)     @        0x105b6f9a6  ray::gcs::ServiceBasedGcsClient::GetGcsServerAddressFromRedis()
(pid=raylet)     @        0x105b73c2e  std::__1::__function::__func<>::operator()()
(pid=raylet)     @        0x105b43319  _ZZN3ray3rpc12GcsRpcClient15ReportHeartbeatERKNS0_22ReportHeartbeatRequestERKNSt3__18functionIFvRKNS_6StatusERKNS0_20ReportHeartbeatReplyEEEEENKUlS9_SC_E_clES9_SC_
(pid=raylet)     @        0x105b42c9b  ray::rpc::ClientCallImpl<>::OnReplyReceived()
(pid=raylet)     @        0x105b711a5  _ZN5boost4asio19asio_handler_invokeIZN3ray3rpc17ClientCallManager29PollEventsFromCompletionQueueEiEUlvE_EEvR